In [2]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
import networkx as nx
import math
import graph_tool.all as gt

In [3]:
rndm_seed = 45

Iniziamo prendendo tre netwrok come scheletro: Karate Club, Florentie Family e Les Miserable Network. Ci costruiamo sopra una rete in cui i pesi degli archi sono presi da una distribuzione normale con media e deviazione standard date.

In [4]:
def build_network(mean=None, std=None, seed=0, network="Karate"):
    """
    Crea una matrice W 'vera' da usare per l'inferenza della rete dopo. Lo faccio usando la topologia del modello scelto:
    -> network == "Karate": N=34, E=78
    -> network == "Miserable": N=77, E=254
    -> network == "Family": N=15, E=20
    
    input param:
    -> mean: media dei pesi assegnati 
    -> std: dev. standard dei pesi assegnati
    -> seed: seed per la randomicita'
    I pesi vengono presi da una normale N(mean, std)
    
    output:
    -> W: matrice dei pesi del network scelto con i pesi gia' assegnati
    """
    rng = np.random.default_rng(seed)
    
    # prendo il grafo nel network scelto
    if network == "Karate":
        G = nx.karate_club_graph()
    elif network == "Miserable":
        G = nx.les_miserables_graph()
    elif network == "Family":
        G = nx.florentine_families_graph()
    elif network == "Football":
        G = nx.read_gml("data/football.gml")
    elif network == "Jazz":
        g = gt.collection.ns["jazz_collab"]
        edges = g.get_edges()
        G = nx.Graph()
        G.add_edges_from((int(u), int(v)) for u, v in edges[:, :2])
    else:
        raise ValueError("network non riconosciuto")
    
    G = nx.convert_node_labels_to_integers(G, ordering="sorted")
    
    N = G.number_of_nodes()
    E = G.number_of_edges()
    
    # se non ho la media da cui campionare i pesi la creo basandomi sui valori del grafo
    if mean is None:
        mean = N / (2 * E)
    if std is None:
        if network == "Football" or network == "Jazz":
            std = 0.1 * mean
        else:
            std = 0.01
        
    # inizializzo la matrice dei pesi a zero
    W = np.zeros((N, N), dtype=float)

    # riempio la matrice dei pesi e metto valori presi da N(mean, std) solo quando esiste l'arco
    # NB! G.edges() è una lista di tuple (i, j) che mi dice quali archi esistono
    for i, j in G.edges():
        w_ij = rng.normal(loc=mean, scale=std)
        # Essendo che la rete non è direzionale, la matrice dei pesi sarà simmetrica
        W[i, j] = w_ij
        W[j, i] = w_ij

    # infine metto a 0 gli elementi sulla diagonale
    for i in range(N):
        W[i, i] = 0
    return W, E
    

In [5]:
def build_data(W, M=1000, beta=0.2, seed=1, h=None,
               model="kinetic",
               initial_sweep=200, sweeps_per_sample=1):
    """
    Genera dati con:
      - model="kinetic": Kinetic Ising (stati {-1,+1}) come nel tuo codice.
      - model="equilibrium": Equilibrium Ising ternario (stati {-1,0,+1})
        campionato via Gibbs/heat-bath.

    Parametri extra per equilibrium:
      - initial_sweep: numero di sweep iniziali (per avvicinarsi all'equilibrio)
      - sweeps_per_sample: sweep tra un campione salvato e il successivo (thinning)
    """
    rng = np.random.default_rng(seed)
    N = W.shape[0]

    if h is None:
        h = np.zeros(N, dtype=float)

    X = np.zeros((N, M + 1), dtype=int)

    if model.lower() == "kinetic":
        # Kinetic Ising model
        s = rng.choice([-1, 1], size=N)
        X[:, 0] = s

        for t in range(M):
            H = h + W @ s
            p = 1 / (1 + np.exp(-2 * beta * H))

            s = (rng.random(N) < p).astype(int)
            s[s == 0] = -1
            X[:, t + 1] = s

        return X

    elif model.lower() == "equilibrium":
        # Equilibrium Ising model ternario (stati {-1,0,+1})
        s = rng.choice([-1, 0, 1], size=N)
        X[:, 0] = s

        def one_sweep(s):
            for i in rng.permutation(N):
                # campo locale escludendo i (assumendo W_ii=0)
                Hi = h[i] + (W[i] @ s) - W[i, i] * s[i]

                a = beta * Hi  # beta nel campo
                
                ea  = np.exp(a)
                ema = np.exp(-a)
                Z   = 1.0 + ea + ema

                p_minus = ema / Z   # P(-1)
                p_zero  = 1.0 / Z   # P(0)
                p_plus  = ea  / Z   # P(+1)

                p = [p_minus, p_zero, p_plus]

                s[i] = rng.choice([-1, 0, 1], p=p)
            return s

        # Faccio andare un po la catena di ising
        for _ in range(max(0, initial_sweep)):
            s = one_sweep(s)

        # campionamento: salva M campioni (più il primo già salvato in X[:,0])
        for t in range(M):
            for _ in range(max(1, sweeps_per_sample)):
                s = one_sweep(s)
            X[:, t + 1] = s

        return X

    else:
        raise ValueError("model deve essere 'kinetic' oppure 'equilibrium'")

In [6]:
def plot_weighted_network(W, threshold=0.0, title=None, seed=0, istrue=False,
                          name=None, pos=None, return_pos=False, W_true=None,
                          true_threshold=None):
    """
    Disegna un network pesato.

    Se istrue=False e W_true è fornita:
      - blu  = edge presente sia in W che in W_true (corretto)
      - rosso = edge presente in W ma NON in W_true (sbagliato)
      - giallo = edge presente in W_true ma mancante in W (missing)

    threshold: soglia su |W_ij| per considerare un edge "presente" in W.
    true_threshold: soglia su |W_true_ij| per considerare un edge "presente" in W_true.
                    Se None, usa threshold.
    """

    W = np.asarray(W, dtype=float)
    N = W.shape[0]

    if true_threshold is None:
        true_threshold = threshold

    # Soglia su W
    W_thr = W.copy()
    W_thr[np.abs(W_thr) <= threshold] = 0.0
    np.fill_diagonal(W_thr, 0.0)

    # grafo inferred
    G = nx.from_numpy_array(W_thr)
    E = G.number_of_edges()

    # layout
    if pos is None:
        pos = nx.spring_layout(G, seed=5674932, k=5/np.sqrt(max(N, 1)), scale=2.0)

    # edges inferred e loro pesi
    edges_inf = list(G.edges())
    w_inf = np.array([W_thr[i, j] for i, j in edges_inf], dtype=float)

    # se istrue=True disegna tutto blu (come "vero")
    if istrue or W_true is None:
        if (not istrue) and (W_true is None):
            if E > 0:
                edge_colors = ["red" if np.abs(w - (N / (2 * E))) < 0.1 else "blue" for w in w_inf]
            else:
                edge_colors = ["blue" for _ in w_inf]
        else:
            edge_colors = ["blue" for _ in w_inf]

        abs_w = np.abs(w_inf)
        if abs_w.size > 0 and abs_w.max() > 0:
            edge_widths = 0.5 + 2 * (abs_w / abs_w.max())
        else:
            edge_widths = 1.0

        plt.figure(figsize=(7, 6))
        nx.draw_networkx_nodes(G, pos, node_size=200, node_color="grey", alpha=0.8)
        nx.draw_networkx_edges(G, pos, edge_color=edge_colors, width=edge_widths, alpha=0.8)

        if title is not None:
            plt.title(title)
        plt.axis("off")
        plt.tight_layout()
        if name is not None:
            plt.savefig(name, dpi=300, bbox_inches='tight')
        plt.show()

        if return_pos:
            return pos
        return

    # caso istrue=False e W_true fornita
    W_true = np.asarray(W_true, dtype=float)
    if W_true.shape != W.shape:
        raise ValueError("W_true deve avere la stessa shape di W")

    # set di edge presenti (i<j) per inferred e true
    inf_set = { (i, j) if i < j else (j, i) for (i, j) in edges_inf }

    iup, jup = np.triu_indices(N, k=1)
    true_mask = np.abs(W_true[iup, jup]) > true_threshold
    true_edges = list(zip(iup[true_mask], jup[true_mask]))
    true_set = set(true_edges)

    correct = inf_set & true_set        # blu
    wrong   = inf_set - true_set        # rosso
    missing = true_set - inf_set        # giallo

    # helper per widths coerenti su tutti gli edge disegnati
    def edge_abs_weights(edge_list, M):
        if len(edge_list) == 0:
            return np.array([], dtype=float)
        return np.array([abs(M[i, j]) for (i, j) in edge_list], dtype=float)

    correct_list = sorted(correct)
    wrong_list   = sorted(wrong)
    missing_list = sorted(missing)

    abs_correct = edge_abs_weights(correct_list, W_thr)   # usa peso inferred
    abs_wrong   = edge_abs_weights(wrong_list,   W_thr)   # usa peso inferred
    abs_missing = edge_abs_weights(missing_list, W_true)  # usa peso true

    max_abs = 0.0
    for arr in (abs_correct, abs_wrong, abs_missing):
        if arr.size > 0:
            max_abs = max(max_abs, float(arr.max()))
    if max_abs <= 0:
        max_abs = 1.0

    def scale_width(abs_arr):
        if abs_arr.size == 0:
            return 1.0
        return 0.5 + 2 * (abs_arr / max_abs)

    w_correct = scale_width(abs_correct)
    w_wrong   = scale_width(abs_wrong)
    w_missing = scale_width(abs_missing)

    # Disegno
    plt.figure(figsize=(7, 6))
    nx.draw_networkx_nodes(G, pos, node_size=200, node_color="grey", alpha=0.8)

    width_scale=0.9
    # disegna per gruppi (così colori diversi + missing inclusi)

    if len(wrong_list) > 0:
        nx.draw_networkx_edges(G, pos, edgelist=wrong_list,   edge_color="red",   width=width_scale*w_wrong,   alpha=0.8)
    if len(correct_list) > 0:
        nx.draw_networkx_edges(G, pos, edgelist=correct_list, edge_color="blue",  width=w_correct, alpha=0.9)
    if len(missing_list) > 0:
        nx.draw_networkx_edges(G, pos, edgelist=missing_list, edge_color="orange",  width=2*width_scale*w_missing, alpha=0.9)

    if title is not None:
        plt.title(title)
    plt.axis("off")
    plt.tight_layout()
    if name is not None:
        plt.savefig(name, dpi=300, bbox_inches='tight')
    plt.show()

    if return_pos:
        return pos


In [7]:
def binarize_matrix(A, treshold):
    """
    Data la matrice A restituisce la sua binarized form, ovvero i valori maggiori della treshold (in valore assoluto) diventano 1, e i valori minori diventano 0
    """
    return (np.abs(A) > treshold).astype(int)

In [8]:
def jaccard_similarity(A, B):
    """
    Calcola il valore di Jaccard similarity tra due matrici A e B qualsiasi.
    """
    i, j = np.triu_indices_from(A, k=1)
    Aij = A[i, j]
    Bij = B[i, j]

    num = np.sum(np.abs(Aij - Bij))
    den = np.sum(np.abs(Aij) + np.abs(Bij))
    if den == 0:
        return 1.0
    return 1 - num/den

In [9]:
def logcosh(x):
    """ 
    Esegue log(cosh(x)) in modo stabile.
    """
    return np.logaddexp(x, -x) - np.log(2.0)

In [10]:
def log_factorial(n: int):
    """
    Calcola il logaritmo del fattoriale di un intero n usando la funzione gamma (stabile per n grandi)
    """
    return math.lgamma(n + 1)

In [11]:
def log_binom(n: int, k: int):
    """
    Calcola il logaritmo del coefficiente binomiale \binom{n}{k} in modo numericamente stabile.
    """
    if k < 0 or k > n:
        return float("-inf")
    return math.lgamma(n + 1) - math.lgamma(k + 1) - math.lgamma(n - k + 1)

In [12]:
# Costruisco una funzione che mi quantizza un valore ad un multiplo di Delta

def quantize_to_delta(w: float, Delta: float):
    """
    Quantizza il valore di w ad un multiplo di Delta.
    """
    wq = Delta * np.round(np.asarray(w, dtype=float) / Delta)
    if np.ndim(w) == 0:
        return float(wq)
    return wq

In [13]:
# Ora costruisco una classe che rappresentera' lo stato della matrice per ogni istante della ricostruzione

@dataclass
class W_state:
    X: np.ndarray    # Matrice di dati (N, M+1)   
    h: np.ndarray = None    # Campo esterno locale per ogni nodo (N,) vettore riga
    W: np.ndarray = None   # Matrice dei pesi attuale (N, N), simmetrica e con diagonale 0
    beta: float = 1 # Parametro di 'temperatura' del modello di Ising
    
    S: np.ndarray = None   # E' il vettore s(t), ovvero le colonne 0,...,M di X
    Y: np.ndarray = None   # E' il vettore s(t+1), ovvero le colonne 1,...,M+1 di X
    H: np.ndarray = None   # E' la matrice del campo locale per ogni nodo ad ogni tempo (N,M)

    wmin: float = -0.5     # Valore minimo ammesso per un peso
    wmax: float = 0.5      # Valore massimo ammesso per un peso
    z_vals: list = None    # Lista dei possibili pesi
    Delta: float = 1e-8    # 'Precisione' dei pesi

    tol: float = None      # Tolleranza per capire quando due pesi sono effettivamente diversi da zero

    lam: float = 1        # Iperparametro usato per il priore P(W| λ, Δ)

    def __post_init__(self):
        
        N, Mp1 = self.X.shape
        if self.h is None:
            self.h = np.zeros(N, dtype=float)

        np.fill_diagonal(self.W, 0.0)
        
        self.S = self.X[: , :-1].astype(int)
        self.Y = self.X[: , 1:].astype(int)

        self.H = self.h[:, None] + self.W @ self.S

        if self.z_vals is None:
            self.z_vals = []

        if self.tol is None:
            self.tol = self.Delta / 2
        

    def is_in_z(self, w):
        return any(abs(w-z) <= self.tol for z in self.z_vals) # Mi dice se un valore si trova dentro z_vals

    def add_to_z(self, w):
        wq = quantize_to_delta(w, self.Delta) # Aggiunge un peso alla lista z_vals
        if wq == 0.0:
            return

        for z in self.z_vals:
            if abs(z - wq) <= self.tol:
                return
        self.z_vals.append(wq)
        self.z_vals.sort()

    def refresh_z_vals(self):
        W, tol, Delta = self.W, self.tol, self.Delta
        N = W.shape[0]
        
        iup, jup = np.triu_indices(N, k=1)
        w = W[iup, jup]

        active = np.abs(w) > tol
        if not np.any(active):
            self.z_vals = []
            return

        wq = Delta * np.round(w[active] / Delta)
        wq = wq[np.abs(wq) > tol]
        
        self.z_vals = np.unique(wq).astype(float).tolist()
        self.z_vals.sort()
        

Ora creo delle funzioni che mi calcolano la variazione di description lenght. Queste mi serviranno dopo per poter accettare o meno un certo cabiamento.

1) delta_log_likelihood(state: W_state, i:int, j:int, w_new: float, model="KineticIsing")
2) delta_complexity(state: W_state, i:int, j: int, w_new: float)
3) delta_MDL(state: W_state, i:int, j: int, w_new: float)

In [14]:
# Ora costruisco la funziona che mi calcola la variazione della loglikelihood quando un valore di peso viene cambiato

def delta_log_likelihood(state: W_state, i:int, j:int, w_new: float, model="KineticIsing") -> float:
    """
    Calcola la variazione di loglikelihood dei dati dovuta a una modifica del peso Wij con w_new.
    
    input params:
    -> state: stato corrente di tipo W_state definito sopra
    -> i, j: indici del peso da cambiare
    -> w_new: il nuovo valore del peso da testare
    -> model: selezione del modello generativo dei dati
    
    output params:
    -> ritorna la variazione di loglikelihood dei dati dovuta a una modifica del peso Wij con w_new.
    NB! questa funzione NON cambia il valore del peso, calcola solo quanto varierebbe la loglikelihood
    """
    w_old = state.W[i, j]
    delta = w_new - w_old
    if abs(delta) < state.tol:
        return 0

    # Valori comodi per il calcolo della variazione della loglikelihood
    si, sj = state.S[i], state.S[j]
    yi, yj = state.Y[i], state.Y[j]
    Hi, Hj = state.H[i], state.H[j]
    
    # Aggiorno i valori del campo locale
    Hi_new = Hi + delta * sj
    Hj_new = Hj + delta * si

    # Calcolo la variazione della loglikelihood dovuta a questa variazione di Hi e Hj

    if model == "KineticIsing":
        # remind che per modello di ising Δℓ_n(t) = beta * y_n(t) * ΔH_n - [log(cosh(beta*H'_n(t))) -  log(cosh(beta*H_n(t)))], con H'_n(t) = H_n(t) + ΔH_n(t)
        # quindi calcolo dli e dlj come variazione della loglik ad ogni tempo t
        dli = state.beta * yi * delta * sj - ( logcosh(state.beta * Hi_new) - logcosh(state.beta * Hi) )
        dlj = state.beta * yj * delta * si - ( logcosh(state.beta * Hj_new) - logcosh(state.beta * Hj) )
    return float(np.sum(dli+dlj))

In [15]:
# Funzione che mi calola il valore di log P(W | λ, Δ)

def log_prior(state: W_state):
    """
    Calcola il valore del log-priore log P(W | λ, Δ) della matrice dei pesi corrente,
    secondo il prior discreto usato nell'approccio MDL.
    
    Il calcolo considera:
    - il numero di archi attivi E (pesi diversi da zero)
    - le categorie di peso distinte z_k (ottenute quantizzando con Delta)
    - le molteplicità m_k di ciascuna categoria
    - i termini combinatori e di penalizzazione del prior
    
    input params:
    -> state: stato corrente di tipo W_state, da cui prende:
              - state.W (matrice dei pesi)
              - state.tol (tolleranza per identificare gli zeri)
              - state.Delta (quantizzazione dei pesi)
              - state.lam (iperparametro λ del prior)
    
    output params:
    -> ritorna il valore di log P(W | λ, Δ) come float
    NB!
    -> La funzione usa solo la parte sopra-diagonale della matrice (i<j), perché la rete è undirected.
    -> Se non ci sono archi attivi (E = 0), gestisce separatamente il caso e ritorna -log(B+1), con B = binom(N,2).
    -> Le categorie di peso sono costruite quantizzando i pesi attivi ai multipli di Delta.
    """
    W = state.W
    tol = state.tol
    Delta = state.Delta
    N = W.shape[0]
    lam = state.lam
    # Cerco gli archi diversi da zero
    iup, jup = np.triu_indices(N, k=1)
    w = W[iup, jup]
    
    active_mask = np.abs(w) > tol
    w_active = w[active_mask]
    E = int(w_active.size)

    B = N * (N - 1) // 2 # Comodo da avere: binom(N 2)
    
    # Considero separatamente il caso E == 0
    if E == 0:
        # In questo caso il numeratore e' pari a 1, e al denominatore e' tutto pari a 1 tranne (N 2) + 1 -> rimane solo -log(B + 1)
        return -math.log(B + 1.0)
    
    
    # Ora mi prendo le categorie
    q = np.rint(w_active / Delta).astype(int)
    q = q[np.abs(q) > 0]
    uq, counts = np.unique(q, return_counts=True)
    counts = counts.astype(int)
    K = int(uq.size)
    z_all = uq.astype(float) * Delta
    
    # Calcolo il numeratore
    log_num = 0
    
    log_num += float(np.sum([log_factorial(int(mk)) for mk in counts])) # ∏_k m_k!
    log_num += - lam * float(np.sum(np.abs(z_all))) # exp(-λ Σ_k |z_k|)
    log_num += K * math.log(math.expm1(Delta*lam)) #  (e^{Δλ}-1)^K

    # Calcolo il denominatore
    log_den = 0

    log_den += log_factorial(E)  # E!
    log_den += log_binom(E - 1, K - 1) # binom(E-1  K-1)
    log_den += K * math.log(2.0) # 2^K
    log_den += math.log(E) # E' il termine max(E,1), che qua e' sempre E perche abbiamo gia gestito E == 0
    log_den += log_binom(B, E)   # binom(binom(N 2) E)
    log_den += math.log(B + 1.0) # binom(N 2) + 1
    
    # Unisco i due pezzi
    logP = log_num - log_den
    return logP
    

Poi serve una funzione che applica una variazione di peso se questa viene accettata.

In [16]:
def apply_changes(state: W_state, changes):
    """
    Applica una lista di cambiamenti ai pesi della matrice W.
    """
    for i, j, w_new in changes:
        apply_edge_change(state, int(i), int(j), float(w_new))

In [17]:
# Funzione che applica la variazione di peso alla matrice dei pesi

def apply_edge_change(state: W_state, i:int, j:int, w_new: float) -> None:
    """
    Applica la variazione di un peso alla matrice dei pesi.
    
    La funzione:
    - quantizza prima il nuovo valore w_new a un multiplo di Delta
    - aggiorna la matrice dei pesi in modo simmetrico (rete undirected)
    - aggiorna i campi locali state.H dei due nodi coinvolti
    - mantiene quindi lo stato coerente senza dover ricalcolare H da zero
    
    input params:
    -> state: stato corrente di tipo W_state definito sopra
    -> i, j: indici del peso da cambiare
    -> w_new: il nuovo valore del peso da testare
    
    output params:
    -> nessun output (la funzione modifica direttamente state)
    
    NB!
    -> Se la variazione |w_new - w_old| è minore della tolleranza state.tol, la funzione non applica alcun cambiamento.
    """
    w_new = quantize_to_delta(w_new, state.Delta)
    w_old = state.W[i, j]
    delta = w_new - w_old
    if abs(delta) < state.tol:
        return
    # Aggiorno la matrice dei pesi
    state.W[i, j] = w_new
    state.W[j, i] = w_new
    # Aggiorno il valore di campo locale
    state.H[i] += delta * state.S[j]
    state.H[j] += delta * state.S[i]

In [18]:
# Costruisco la funzione che mi dice il 'costo' in complessita' del network quando un valore di peso viene cambiato

def delta_complexity(state: W_state, changes) -> float:
    """
    Calcola la variazione della complessità (termine di prior) dovuta a una modifica
    simultanea di uno o più pesi della matrice W.
    
    La funzione:
    - calcola il valore di log P(W | λ, Δ) prima della modifica
    - applica temporaneamente i cambiamenti proposti
    - ricalcola log P(W | λ, Δ) dopo la modifica
    - ripristina lo stato iniziale
    - ritorna la differenza tra i due valori
    
    input params:
    -> state: stato corrente di tipo W_state
    -> changes: lista di tuple (i, j, w_new) che rappresentano le modifiche proposte
                ai pesi della matrice W
    
    output params:
    -> ritorna la variazione di complessità come float, cioè:
       Δcomplexity = log P(W_new | λ, Δ) - log P(W_old | λ, Δ)
    
    NB!
    -> La funzione NON lascia applicati i cambiamenti: modifica temporaneamente state
       solo per calcolare la differenza e poi ripristina W, H e z_vals.
    """
    # Mi salvo lo stato iniziale per dopo
    W_old = state.W.copy()
    H_old = state.H.copy()
    z_old = None if state.z_vals is None else state.z_vals.copy()
    
    # Calcolo il valore di log P(W | λ, Δ) prima
    before = log_prior(state)
    
    # Applico le variazioni
    if len(changes) == 1:
        apply_edge_change(state, changes[0][0], changes[0][1], changes[0][2])
    else:
        for a, b, w_new in changes:
            apply_edge_change(state, int(a), int(b), float(w_new))

    # Calcolo il valore di log P(W | λ, Δ) dopo
    after = log_prior(state)

    # Ripristino lo stato iniziale
    state.W[:] = W_old
    state.H[:] = H_old
    state.z_vals = z_old
    #print(before)
    return after - before

Mi serve una funzione che mi calcoli la variazione di loglikelihood quando piu edges che includono nodi comuni vengono cambiati.
Ricordo che nel modello di Ising cinetico, la log-likelihood dipende dai campi locali
$H_n(t)$, definiti come

$$
H_n(t) = h_n + \sum_j W_{nj}\, s_j(t).
$$

Quando si modifica un singolo peso $W_{ab}$ di una quantità
$\Delta W_{ab} = w_{\text{new}} - w_{\text{old}}$, i campi locali
dei nodi coinvolti cambiano come segue:

$$
\Delta H_a(t) = \Delta W_{ab}\, s_b(t),
\qquad
\Delta H_b(t) = \Delta W_{ab}\, s_a(t),
$$

mentre i campi di tutti gli altri nodi rimangono invariati.

Nel caso in cui vengano applicate **più modifiche ai pesi simultaneamente**, la variazione del campo
locale di un nodo è data dalla **somma di tutti i contributi** associati ai
pesi incidenti su quel nodo:

$$
\Delta H_n(t) = \sum_{(n,j)\in\mathcal{C}} \Delta W_{nj}\, s_j(t),
$$

dove $\mathcal{C}$ è l’insieme delle modifiche proposte.

Poiché la log-likelihood non è una funzione lineare di $H_n(t)$,
la variazione totale non può essere calcolata come somma di variazioni
indipendenti se le modifiche condividono nodi.
È quindi necessario calcolare la variazione combinata della log-likelihood
utilizzando direttamente i campi aggiornati $H_n(t) + \Delta H_n(t)$.


In [19]:
# Creo una funzione che mi calcola la variazione di loglikelihood per una variazione simultanea di piu pesi

def delta_multi_log_likelihood(state: W_state, changes):
    """
    Calcola la variazione di loglikelihood dovuta a una variazione simultanea
    di più pesi della matrice W. In questo caso, le variazioni dei campi locali H_n(t) per ciascun nodo
    vengono sommate prima di valutare la variazione della loglikelihood

    Per ciascun nodo coinvolto, viene costruita la variazione totale del campo
    locale ΔH_n(t), e la variazione di loglikelihood è calcolata come:

        Δℓ_n(t) = β y_n(t) ΔH_n(t)
                  - [ logcosh(β(H_n(t)+ΔH_n(t))) - logcosh(β H_n(t)) ]

    input params:
    -> state: stato corrente di tipo W_state
    -> changes: lista di tuple (a, b, w_new) che rappresentano le variazioni
                simultanee dei pesi W_ab → w_new (con a < b)

    output params:
    -> DLL: variazione totale della loglikelihood dovuta alle modifiche proposte
    """
    
    deltaH = {}

    for a, b, w_new in changes:
        a = int(a)
        b = int(b)
        if a == b:
            continue
        if a > b:
            a, b = b, a

        w_old = float(state.W[a, b])
        dw = float(w_new - w_old)
        if abs(dw) < state.tol:
            continue

        #  Se il nodo corrente non e' mai stato incontrato creo il vettore del campo H agente su quel nodo ad ogni tempo
        if a not in deltaH:
            deltaH[a] = np.zeros(state.S.shape[1], dtype=float)
        if b not in deltaH:
            deltaH[b] = np.zeros(state.S.shape[1], dtype=float)

        # Mi salvo tutte le variazioni ad ogni tempo
        deltaH[a] += dw * state.S[b]
        deltaH[b] += dw * state.S[a]

    if len(deltaH) == 0:
        return 0.0

    DLL = 0.0

    for n, dHn in deltaH.items():
        Hn = state.H[n]
        Yn = state.Y[n]
        Hn_new = Hn + dHn
        # Trovo la variazione di loglikelihood ad ogni tempo nel vettore dln
        # Δℓ_n(t) = beta * y_n(t) * ΔH_n(t) - [logcosh(beta(H+ΔH)) - logcosh(beta H)]
        dln = state.beta * Yn * dHn - (logcosh(state.beta * Hn_new) - logcosh(state.beta * Hn))
        # Sommo su tutti i tempi
        DLL += float(np.sum(dln))
    return DLL

In [20]:
# Costruisco una funzione che mi dice quanto cambia il peso totale tra loglikelihood e description length quando un valore di peso viene cambiato

def delta_MDL(state: W_state, changes) -> float:
    """
    Calcolo totale della variazione della Description Length, che include loglikelihood e complexity, se dovesse cambiare un peso Wij.
    
    input params:
    -> state: stato corrente di tipo W_state definito sopra
    -> i, j: indici del peso da cambiare
    -> w_new: il nuovo valore del peso da testare
    
    output params:
    -> il costo totale della variazione della description length
    NB! questa funzione NON cambia il valore del peso, calcola solo quanto varierebbe la description length
    """
    if len(changes) == 1:
        dlog = delta_log_likelihood(state, changes[0][0], changes[0][1], changes[0][2])
    else:
        dlog = delta_multi_log_likelihood(state, changes)
    dprior = delta_complexity(state, changes)
    #print(dlog + dprior)
    return dlog + dprior

**IMPLEMENTAZIONE EDGE UPDATE**

In [21]:
# Creo una funzione che data la lista di pesi z_vals mi dice il valore minimo dei pesi positivi e massimo dei pesi negaitvi

def wmin_wmax(z_vals):
    """
    Restituisce i due valori di categoria più vicini a zero:
    - w_min: il massimo dei valori negativi
    - w_max: il minimo dei valori positivi

    input params:
    -> z_vals: lista delle categorie di peso attualmente presenti

    output params:
    -> w_min, w_max (None se la categoria non esiste)
    """
    if len(z_vals) == 0:
        return None, None
    z_pos = [z for z in z_vals if z>0]
    z_neg = [z for z in z_vals if z<0]
    w_max = min(z_pos) if len(z_pos)>0 else None
    w_min = max(z_neg) if len(z_neg)>0 else None
    return w_min, w_max

In [22]:
# Creo una funzione che mi ritorna una matrice in cui ogni entrata ij corrisponde alla derivata della loglik rispetto a Wij
# per il modello di ising cinetico questo vale Gij = somme su t di (beta * s_j(t) * (s_i(t+1) - tgh(beta * H_i(t)))

def loglik_gradient(state: W_state):
    """
    Calcola il modulo del gradiente della log-likelihood rispetto ai pesi Wij
    per il modello di Ising cinetico.

    input params:
    -> state: stato corrente di tipo W_state

    output params:
    -> matrice G_ij = |∂ log L / ∂ W_ij|, simmetrica
    """
    A = state.Y - np.tanh(state.beta * state.H)
    B = A @ state.S.T
    G = state.beta * (B + B.T) # dato che la matrice e' simmetrica
    return np.abs(G)

In [23]:
def choose_best_candidate_edges(state: W_state, rng, kappa: float = 1.0):
    """
    Seleziona un insieme di archi candidati (i, j) con peso attualmente nullo
    da considerare per l’edge update.

    input params:
    -> state: stato corrente di tipo W_state
    -> rng: generatore di numeri casuali
    -> kappa: fattore che controlla il numero di candidati selezionati (K = ceil(kappa * N))

    output params:
    -> set di tuple (i, j) contenente gli archi candidati

    NB:
    - se z_vals è vuoto, la selezione è basata sul modulo del gradiente della log-likelihood
    - se z_vals non è vuoto, il ranking è basato sul miglior incremento di log-likelihood
      ottenibile provando le categorie più vicine a zero
    """
    
    N = state.W.shape[0]
    K = int(np.ceil(kappa * N))

    # Considero solo gli elementi sopra la diagonale
    iup, jup = np.triu_indices(N, k=1)
    zeros = (np.abs(state.W[iup, jup]) <= state.tol)

    if not np.any(zeros):
        return set()

    # Caso in cui z_values e' vuoto -> uso il gradiente
    if len(state.z_vals) == 0:
        scores_all = loglik_gradient(state)
        scores = scores_all[iup, jup]
        scores = np.where(zeros, scores, -np.inf)

    # Caso in cui z_values non e' vuoto
    else:
        w_minus, w_plus = wmin_wmax(state.z_vals)
        # Inizializzo gli score a -infinito 
        scores = np.full(iup.shape[0], -np.inf, dtype=float)
        # Considero gli edge messi a zero
        idxs = np.where(zeros)[0]

        # Salvo i vari scores dentro un apposita lista
        for idx in idxs:
            i, j = int(iup[idx]), int(jup[idx])

            best = -np.inf
            if w_plus is not None:
                best = max(best, delta_log_likelihood(state, i, j, w_plus))
            if w_minus is not None:
                best = max(best, delta_log_likelihood(state, i, j, w_minus))

            scores[idx] = best
    # Ora prendo i top K valori
    valid = np.isfinite(scores)
    if np.sum(valid) <= K:
        possible_cand = np.where(valid)[0]
    else:
        #Ora seleziono solo gli indici dei K valori di scores piu grandi (argpartition sorta dal piu piccolo al piu grande, quindi uso -K)
        possible_cand = np.argpartition(scores, -K)[-K:]
        possible_cand = possible_cand[np.isfinite(scores[possible_cand])]

    # Infine seleziono i candidati
    cand = set()
    for idx in possible_cand:
        i, j = int(iup[idx]), int(jup[idx])
        cand.add((i, j))
    return cand

In [24]:
# Costruzione dell'insieme E dei candidati per l'edge update

def build_E_list(state: W_state, candidate_edges) -> list:
    """
    Costruisce l’insieme E degli archi su cui applicare l’edge update.

    L’insieme E è dato dall’unione tra:
    - gli archi candidati con peso nullo selezionati dalla funzione choose_best_candidate_edges(state: W_state, rng, kappa: float = 1.0)
    - gli archi con peso attualmente diverso da zero

    input params:
    -> state: stato corrente di tipo W_state
    -> candidate_edges: set di tuple (i, j) degli archi candidati

    output params:
    -> lista di tuple (i, j) contenente gli archi su cui tentare l’edge update
    """
    N = state.W.shape[0]
    Eset = set()
    
    # Aggiungo gli edges candidati
    for (i, j) in candidate_edges:
        if i==j:
            continue
        if i > j:
            i, j = j, i
        if 0 <= i < N and 0 <= j < N:
            Eset.add((i, j))

    # Aggiungo gli edges con peso gia' diverso da zero
    NonZeroEdg = np.argwhere(np.triu(abs(state.W) > state.tol, 1))
    for i, j in NonZeroEdg:
        Eset.add((int(i), int(j)))

    return list(Eset)

In [25]:
# Creo una funzione che propone un nuovo peso che appartiene a z di un dato edge (i, j), lo faccio cercando un valore ottimale con una random bisection search

def new_weight_fromZ_RB(state: W_state, i: int, j: int, rng, nsteps = 10):
    """
    Propone un nuovo valore di peso per l’arco (i, j) scegliendolo tra
    le categorie di peso già esistenti z_vals.

    La ricerca del peso ottimale è effettuata tramite una random bisection
    che massimizza la variazione della MDL.

    input params:
    -> state: stato corrente di tipo W_state
    -> i, j: indici dell’arco da aggiornare
    -> rng: generatore di numeri casuali
    -> nsteps: numero di iterazioni della random bisection

    output params:
    -> best_w: valore di peso candidato (None se z_vals è vuoto)
    -> best_d: variazione massima della MDL associata al peso proposto
    """

    z = state.z_vals
    K = len(z)

    if K == 0:
        return None, float("-inf")

    L, R = 0, K-1

    #Scelgo random un valore tra quelli esistenti
    best_idx = int(rng.integers(L, R+1))
    best_w = z[best_idx]
    change = [(i, j, best_w)]
    best_d = delta_MDL(state, changes=change)

    for _ in range(nsteps):
        if L == R:
            break

        # Scelgo punto a caso
        m = int(rng.integers(L, R+1))
        w = z[m]
        change = [(i, j, w)]
        d = delta_MDL(state, change)
        
        # Lo salvo se migliora
        if d > best_d:
            best_d, best_w, best_idx = d, w, m
        # Restringo l'intervallo
        mid = (L + R) // 2
        if best_idx <= mid:
            R = mid
        else:
            L = mid + 1

    return best_w, best_d

In [26]:
def edge_update_option1(state: W_state, i: int, j: int, rng, nsteps=10):
    """
    Esegue l’opzione 1 dell’edge update: aggiorna il peso W_ij scegliendo
    un valore tra le categorie di peso già esistenti (z_vals).

    La mossa è accettata solo se la variazione della MDL è positiva.

    input params:
    -> state: stato corrente di tipo W_state
    -> i, j: indici dell’arco da aggiornare
    -> rng: generatore di numeri casuali
    -> nsteps: numero di iterazioni della random bisection

    output params:
    -> ok: True se la mossa è state eseguita, False se rifiutata
    -> w_new: valore del peso proposto
    -> gain: variazione della MDL associata alla mossa
    """
    w_new, gain = new_weight_fromZ_RB(state, i, j, rng=rng, nsteps=nsteps)
    if w_new is None:
        return False, None, 0.0

    if gain > 0:
        apply_edge_change(state, i, j, w_new)
        return True, w_new, gain

    return False, w_new, gain

In [27]:
# Ora creo una funzione che propone un nuovo peso che NON appartiene gia' a z di un dato edge (i, j), lo faccio cercando un valore ottimale con una random bisection search

def new_weight_candidate_RB(state: W_state, i:int, j: int, rng, nsteps=10, max_out = 10):
    """
    Propone un nuovo valore di peso per l’arco (i, j) che non appartiene
    alle categorie di peso già esistenti z_vals.

    Il valore candidato è trovato tramite una random bisection search
    che massimizza la variazione della MDL all’interno dell’intervallo
    [wmin, wmax].

    input params:
    -> state: stato corrente di tipo W_state
    -> i, j: indici dell’arco da aggiornare
    -> rng: generatore di numeri casuali
    -> nsteps: numero di iterazioni della random bisection
    -> max_out: numero massimo di tentativi indipendenti

    output params:
    -> best_d_glob: miglior incremento della MDL trovato
    -> best_wq_glob: valore di peso candidato associato
    """

    best_d_glob = float("-inf")
    best_wq_glob = None

    out = 0
    while out < max_out:
        best_wq = None
        best_d = float("-inf")
        out += 1

        valid = 0
        
        a, b = float(state.wmin), float(state.wmax)
        for _ in range(nsteps):
            # Scelgo un punto da cui partire a caso tra a e b
            m = rng.uniform(a, b)
            mq = quantize_to_delta(m, state.Delta)

            if mq == 0.0 or state.is_in_z(mq):
                continue

            valid += 1
            
            # Per quel valore calcolo la variazione di loglikelihood
            change = [(i, j, mq)]
            d = delta_MDL(state, change)

            if not np.isfinite(d):
                continue
            
            # Salvo il valore del peso se la variazione e' positiva e aggiorno l'intervallo
            if d > best_d:
                best_d, best_wq = d, mq
    
                # Una volta fatto restringo l'intervallo attorno al valore scelto
                new_width = 0.5 * (b - a)
                # Aggiorno a e b
                a = max(state.wmin, best_wq - new_width/2)
                b = min(state.wmax, best_wq + new_width/2)

            if b - a < state.tol:
                break
        if valid == 0:
            continue
            
        if best_d > best_d_glob:
            best_d_glob, best_wq_glob = best_d, best_wq
            
    return best_d_glob, best_wq_glob

In [28]:
# Creo la funzione che esegue la seconda opzione dell'edge update

def edge_update_option2(state: W_state, i: int, j:int, rng, nsteps=10):
    """
    Esegue l’opzione 2 dell’edge update: propone un nuovo valore di peso per W_ij
    che non appartiene alle categorie già presenti (z_vals), tramite random bisection
    in [wmin, wmax].

    La mossa è accettata solo se la variazione della MDL è positiva; in caso di accettazione
    la nuova categoria viene aggiunta a z_vals.

    input params:
    -> state: stato corrente di tipo W_state NB! in state ci sono i valori state.wmin, state.wmax
    -> i, j: indici dell’arco da aggiornare
    -> rng: generatore di numeri casuali
    -> nsteps: numero di iterazioni della random bisection

    output params:
    -> ok: True se la mossa è accettata
    -> wq: valore del peso proposto (quantizzato a Delta)
    -> gain: variazione della MDL associata alla mossa
    """

    gain, wq = new_weight_candidate_RB(state, i, j, rng=rng, nsteps=nsteps)
    
    if wq is None:
        return False, None, float("-inf")
        
    if wq == 0.0 or state.is_in_z(wq):
        return False, None, float("-inf")
        
    if not np.isfinite(gain):
        return False, None, float("-inf")

    
    # Accetto se aumenta la loglik
    if gain > 0:
        apply_edge_change(state, i, j, wq)
        state.add_to_z(wq)
        return True, wq, gain
        
    return False, wq, gain

In [29]:
# Creo la funzione che esegue la terza opzione dell'edge update

def edge_update_option3(state: W_state, i: int, j: int):
    """
    Esegue l’opzione 3 dell’edge update: rimuove l’arco (i, j)
    ponendo il peso W_ij = 0.

    La mossa è accettata solo se la variazione della MDL è positiva.

    input params:
    -> state: stato corrente di tipo W_state
    -> i, j: indici dell’arco da rimuovere

    output params:
    -> ok: True se la mossa è accettata
    -> w_new: valore del peso dopo l’update (0.0)
    -> gain: variazione della MDL associata alla mossa
    """
    w_new = 0.0
    change = [(i, j, w_new)]
    gain = delta_MDL(state, change)

    if gain > 0:
        apply_edge_change(state, i, j, w_new)
        return True, 0.0, gain
    return False, 0.0, gain

In [30]:
# Creo la funzione che appica a caso una delle tre opzioni di edge update ad un dato peso

def random_edge_update(state: W_state, i: int, j: int, rng):
    """
    Applica casualmente una delle tre opzioni di edge update all’arco (i, j),
    scegliendo solo tra le mosse ammissibili nello stato corrente.

    Le opzioni disponibili sono:
    - opzione 1: riassegna il peso usando categorie già esistenti (z_vals)
    - opzione 2: propone una nuova categoria di peso
    - opzione 3: rimuove l’arco ponendo W_ij = 0 (solo se l’arco è presente)

    input params:
    -> state: stato corrente di tipo W_state
    -> i, j: indici dell’arco da aggiornare
    -> rng: generatore di numeri casuali

    output params:
    -> ok: True se la mossa è accettata
    -> w_new: valore del peso proposto
    -> gain: variazione della MDL associata alla mossa
    """
    options = []
    
    if len(state.z_vals) > 0:
        options.append(1)

    options.append(2)

    if abs(state.W[i, j]) > state.tol:
        options.append(3)

    opt = rng.choice(options)

    if opt == 1:
        return edge_update_option1(state, i, j, rng)
    if opt == 2:
        return edge_update_option2(state, i, j, rng)
    if opt == 3:
        return edge_update_option3(state, i, j)    

In [31]:
def edge_update_sweep(state: W_state, E_list, rng):
    """
    Esegue uno sweep di edge update sull’insieme di archi E_list.

    Per ciascun arco (i, j) in E_list viene applicata casualmente
    una delle tre opzioni di edge update ammissibili.

    input params:
    -> state: stato corrente di tipo W_state
    -> E_list: lista di archi (i, j) su cui applicare l’edge update
    -> rng: generatore di numeri casuali

    output params:
    -> accepted: numero totale di mosse accettate nello sweep
    """
    
    accepted = 0
    rng.shuffle(E_list)
    # Applico l'edge update casuale ad ogni (i, j) trovato
    for i, j in E_list:
        ok, w_new, gain = random_edge_update(state, i, j, rng)
        accepted += int(ok)
    return accepted

**IMPLEMENTAZIONE EDGE MOVE**

In [32]:
def edge_move_fornode(state: W_state,i: int, E_list, rng, max_tries: int = 20):
    """
    Prova una edge-move fissando il nodo i (swap locale di un singolo endpoint).

    Procedura:
    - sceglie j tale che (i, j) ∈ E_list e W_ij = 0
    - sceglie u tale che W_iu != 0
    - propone: W_ij <- W_iu  e  W_iu <- 0
    - accetta solo se ΔMDL > 0

    input params:
    -> state: stato corrente di tipo W_state
    -> i: nodo fissato
    -> E_list: lista di archi candidati (insieme E)
    -> rng: generatore di numeri casuali
    -> max_tries: numero massimo di tentativi di proposta

    output params:
    -> ok: True se la mossa è accettata
    -> move: tupla (i, j, u) con j target (zero) e u sorgente (non-zero)
    -> gain: variazione della MDL associata alla mossa
    """

    N = state.W.shape[0]

    # Creo la lista con le posizioni in E_list in cui i pesi valgono zero
    zeros_in_Elist = []
    for a, b in E_list:
        a = int(a)
        b = int(b)
        
        if a == b:
            continue
        if a == i:
            j = b
        elif b == i:
            j = a
        else:
            continue
        if j == i:
            continue

        if abs(state.W[i, j]) <= state.tol:
            zeros_in_Elist.append(j)

    if len(zeros_in_Elist) == 0:
        return False, None, 0.0

    # Creo una lista in cui ci sono gli indici in cui state.W[i] e' diverso da zero
    nz_list = np.flatnonzero(np.abs(state.W[i]) > state.tol)
    # Tolgo l'elemento W[i, i]
    nz_list = nz_list[nz_list != i]

    # Se non ci sono vicini diversi da zero vado al prossimo ciclo
    if len(nz_list) == 0:
        return False, None, 0.0


    # Ora faccio un ciclo per avere piu tentativi se non si dovesse subito riuscire a trovare un i che abbia un vicino diverso da zero
    for _ in range(max_tries):

        j = int(rng.choice(zeros_in_Elist))
        u = int(rng.choice(nz_list))

        if u == j:
            continue
        
        # Quindi ora salvo il peso da spostare
        w = float(state.W[i, u])

        # Calcolo il guadagno della loglik
        changes = [(i, u, 0.0), (i, j, w)]
        gain = delta_MDL(state, changes)
        # E accetto solo se migliora
        if gain>0:
            apply_edge_change(state, i, u, 0.0)
            apply_edge_change(state, i, j, w)
            return True, (i, j, u), gain
    # Se non trovo una mossa valida rinuncio
    return False, None, 0.0

In [33]:
def edge_move_sweep(state: W_state, E_list, rng, max_tries_pernode: int = 20):
    """
    Esegue uno sweep di edge-move sui nodi con grado non nullo.

    Per ciascun nodo i con almeno un arco attivo, viene tentata una
    edge-move (swap locale) fissando il nodo i e usando lo stesso insieme E.

    input params:
    -> state: stato corrente di tipo W_state
    -> E_list: lista di archi candidati (insieme E)
    -> rng: generatore di numeri casuali
    -> max_tries_pernode: numero massimo di tentativi per nodo

    output params:
    -> acc: numero di mosse accettate
    -> tried: numero di nodi su cui è stata tentata la mossa
    """

    # Guardo quali nodi hanno dei vicini non zero
    degree = np.sum(np.abs(state.W) > state.tol, axis=1)
    nodes = np.where(degree > 0)[0] # Qui prendo tuti i valori di i per cui Wij e' diverso da zero, ovvero quando il nodo i ha un edge non nullo con j

    if len(nodes) == 0:
        return 0, 0

    nodes = nodes.copy()
    rng.shuffle(nodes)

    acc = 0
    tried = 0

    for i in nodes:
        tried += 1
        accepted, _, _ = edge_move_fornode(state, int(i), E_list, rng, max_tries_pernode)
        acc += int(accepted)
    return acc, tried

**IMPLEMENTAZIONE EDGE SWAP**

In [34]:
def propose_edge_swap(state: W_state, rng, max_tries: int = 50):
    """
    Propone una mossa di edge swap tra due archi attivi scelti casualmente.
    
    La funzione:
    - seleziona gli archi non nulli della matrice dei pesi (solo parte sopra-diagonale)
    - estrae a caso due archi distinti
    - verifica che coinvolgano 4 nodi distinti
    - costruisce una delle due possibili ricombinazioni topologiche
    - controlla che i due nuovi archi siano attualmente nulli
    - restituisce la lista dei cambiamenti nel formato richiesto da delta_multi_log_likelihood
    
    input params:
    -> state: stato corrente di tipo W_state
    -> rng: generatore di numeri casuali
    -> max_tries: numero massimo di tentativi per trovare una proposta di swap valida
    
    output params:
    -> ritorna una tupla (ok, changes) dove:
       - ok: True se è stata trovata una proposta valida, False altrimenti
       - changes: lista di tuple (i, j, w_new) con i cambiamenti da applicare
                  (oppure None se non viene trovata una proposta valida)
    
    NB!
    -> La mossa proposta è solo topologica: i due pesi originali vengono spostati su due nuovi archi,
       mentre i due archi iniziali vengono posti a zero.
    -> La funzione NON applica lo swap allo stato: costruisce solo la proposta.
    """

    # Seleziono gli archi non zero
    non_zeros = np.argwhere(np.triu(np.abs(state.W) > state.tol, 1))
    
    if len(non_zeros) < 2:
        return False, None

    for _ in range(max_tries):
        # Pesco a caso due archi
        idx1, idx2 = rng.choice(len(non_zeros), size=2, replace=False)
        i, j = int(non_zeros[idx1][0]), int(non_zeros[idx1][1])
        u, v = int(non_zeros[idx2][0]), int(non_zeros[idx2][1])

        # Verifico innanzitutto se ho preso 4 nodi distinti
        if len({i, j, u, v}) < 4:
            continue

        w_ij = float(state.W[i, j])
        w_uv = float(state.W[u, v])

        # Scelgo a caso una delle due possibili combinazioni
        if rng.random() < 0.5:
            # (i,j),(u,v) -> (i,v),(u,j)
            a1, b1 = i, v
            a2, b2 = u, j
        else:
            # (i,j),(u,v) -> (i,u),(j,v)
            a1, b1 = i, u
            a2, b2 = j, v

        # Ordino le coppie, tanto la matrice e' simmetrica
        if a1 > b1: a1, b1 = b1, a1
        if a2 > b2: a2, b2 = b2, a2

        # Vado avanti solamente se i due nuovi archi sono entrambi zero, dato che la mossa swap e' solo di tipo topologica
        if abs(state.W[a1, b1]) > state.tol:
            continue
        if abs(state.W[a2, b2]) > state.tol:
            continue

        # Salvo gli indici vecchi
        ii, jj = (i, j) if i < j else (j, i)
        uu, vv = (u, v) if u < v else (v, u)

        # Salvo i due archi nel formato che vuole la funzione delta_multi_log_likelihood, ovvero una lista di tuple (i, j, w_new)
        changes = [
            (ii, jj, 0.0),
            (uu, vv, 0.0),
            (a1, b1, w_ij),
            (a2, b2, w_uv)
        ]
        return True, changes

    return False, None
        
        

In [35]:
def edge_swap_sweep(state: W_state, rng, n_swaps: int = 20, max_tries: int = 50):
    """
    Esegue uno sweep di edge swaps.

    Per n_swaps volte:
    - propone uno swap con propose_edge_swap()
    - calcola il guadagno Δ
    - accetta e applica i cambi solo se gain > 0

    input params:
    -> state: stato corrente di tipo W_state
    -> rng: generatore di numeri casuali
    -> n_swaps: numero di proposte di swap da tentare
    -> max_tries: numero massimo di tentativi interni per trovare una proposta valida

    output params:
    -> acc: numero di swap accettati
    -> tried: numero di swap effettivamente proposti
    """
    acc = 0
    tried = 0

    for _ in range(n_swaps):
        ok, changes = propose_edge_swap(state, rng, max_tries=max_tries)
        if not ok:
            continue

        tried += 1

        # Dato che la mossa swap e' solo topologica la complessita' non cambia quindi ci interessa solo la Δloglik
        gain = delta_multi_log_likelihood(state, changes)
        
        # # Debug
        # print(gain)
        
        if gain > 0:
            for a, b, w_new in changes:
                apply_edge_change(state, a, b, float(w_new))
            acc += 1
    return acc, tried
    

**IMPLEMENTAZIONE WEIGHT CATEGORY VALUE CHANGE**

In [36]:
# Definisco una funzione che mi ritorna tutti i pesi con un determinato valore

def edges_with_spec_value(state: W_state, zk: float):
    """
    Ritorna lista di (i, j) (i < j) tali che W_ij = zk.
    """
    idxs = np.argwhere(np.triu(np.abs(state.W - zk) <= state.tol, k=1))
    return [(int(i), int(j)) for i, j in idxs]

In [37]:
# Ora una funzione che mi applica piu cambiamenti simultaneamente

def apply_category_value_change(state: W_state, edges_k, z_new: float):
    for (i, j) in edges_k:
        apply_edge_change(state, i, j, z_new)

In [38]:
def propose_new_category_weight_RB(state: W_state, edges_k, rng, nsteps=12, max_out=5):
    """
    Propone un nuovo valore di categoria z_new per un insieme di archi edges_k,
    cercandolo con una random bisection search nell'intervallo [wmin, wmax].
    
    La funzione:
    - estrae un valore iniziale casuale z0 in [wmin, wmax] e lo quantizza a Delta
    - valuta il guadagno di MDL assegnando z0 a tutti gli archi in edges_k
    - ripete una random bisection search per nsteps passi
    - restringe progressivamente l'intervallo attorno al miglior valore trovato
    - ripete la procedura per max_out tentativi indipendenti
    - restituisce il miglior valore z_new e il relativo guadagno di MDL
    
    input params:
    -> state: stato corrente di tipo W_state
    -> edges_k: lista di archi (i, j) a cui assegnare simultaneamente lo stesso valore di peso
    -> rng: generatore di numeri casuali
    -> nsteps: numero di iterazioni della random bisection per ciascun tentativo
    -> max_out: numero di tentativi indipendenti della ricerca
    
    output params:
    -> ritorna una tupla (best_z, best_gain) dove:
       - best_z: miglior valore di categoria trovato (quantizzato a Delta)
       - best_gain: miglior guadagno di MDL associato
    -> se edges_k è vuoto, ritorna (None, -inf)
    
    NB!
    -> La funzione NON applica il cambiamento a state: valuta solo proposte tramite delta_MDL.
    -> Tutti gli archi in edges_k vengono aggiornati insieme allo stesso valore z_new.
    """
    if len(edges_k) == 0:
        return None, float("-inf")

    best_gain = float("-inf")
    best_z = None

    for _ in range(max_out):
        a, b = float(state.wmin), float(state.wmax)

        # Punto iniziale
        z0 = quantize_to_delta(rng.uniform(a, b), state.Delta)
        # Tengo solo i pesi diversi da zero
        if abs(z0) <= state.tol:
            continue
        
        changes0 = [(i, j, z0) for (i, j) in edges_k]
        gain0 = delta_MDL(state, changes0)
        
        # Assegno il punto iniziale solo se migliora
        if np.isfinite(gain0) and gain0 > best_gain:
            best_gain, best_z = gain0, z0

        # Ora effettuo la Bisection Search
        for _ in range(nsteps):
            m = rng.uniform(a, b)
            z = quantize_to_delta(m, state.Delta)
            if abs(z) <= state.tol:
                continue

            # Calcolo il cambiamento di MDL
            changes = [(i, j, z) for (i, j) in edges_k]
            g = delta_MDL(state, changes)

            if not np.isfinite(g):
                continue

            # Accetto e restringo se il guadagno e' migliore dell ultimo tentativo
            if g > best_gain:

                best_gain, best_z = g, z

                width = 0.5 * (b - a)
                a = max(state.wmin, best_z - width/2)
                b = min(state.wmax, best_z + width/2)

            if (b - a) < state.tol:
                break
    return best_z, best_gain

In [39]:
# Infine la funzione che effettivamente applica l'aggiornamento di categoria su tutte le categorie presenti

def weight_category_value_sweep(state: W_state, rng, nsteps=12, max_out=6):
    """
    Esegue uno sweep di aggiornamento del valore delle categorie di peso correnti.
    
    La funzione:
    - aggiorna prima la lista delle categorie presenti con state.refresh_z_vals()
    - itera su tutte le categorie zk correnti
    - per ciascuna categoria trova tutti gli archi con peso zk
    - propone un nuovo valore z_new con una random bisection search
    - accetta la modifica solo se il guadagno di MDL è positivo
    - applica il cambiamento simultaneamente a tutti gli archi della categoria
    - aggiorna di nuovo z_vals dopo ogni mossa accettata
    
    input params:
    -> state: stato corrente di tipo W_state
    -> rng: generatore di numeri casuali
    -> nsteps: numero di iterazioni della random bisection per ogni categoria
    -> max_out: numero di tentativi indipendenti per la ricerca del nuovo valore di categoria
    
    output params:
    -> ritorna una tupla (acc, tried) dove:
       - acc: numero di aggiornamenti di categoria accettati
       - tried: numero di categorie su cui è stato tentato l'aggiornamento
    
    NB!
    -> La funzione non accetta una proposta se z_new coincide con una categoria già esistente
       diversa da quella corrente zk.
    """

    state.refresh_z_vals()
    z_list = state.z_vals.copy()

    acc = 0
    tried = 0
    for zk in z_list:
        tried += 1
        edges_k = edges_with_spec_value(state, zk)
        z_new, gain = propose_new_category_weight_RB(
            state,
            edges_k=edges_k,
            rng=rng,
            nsteps=nsteps,
            max_out=max_out
        )
        if z_new is None:
            continue
        z_new = float(z_new)
        
        if state.is_in_z(z_new) and abs(z_new - zk) > state.tol:
            continue

        if gain > 0 and abs(z_new - zk) > state.tol:
            apply_category_value_change(state, edges_k, z_new)
            acc += 1
            state.refresh_z_vals()
    return acc, tried     

**IMPLEMENTAZIONE WEIGHT CATEGORY DISTRIBUTION**

In [41]:
# Funzione che prova la mossa merge tra due categorie di peso

def merge_weight_category_move(state: W_state, rng, nsteps=12, max_out=5, max_tries=20, accept_zero=False):
    """
    Prova una mossa di merge tra due categorie di peso scelte casualmente.
    
    La funzione:
    - aggiorna prima la lista delle categorie presenti con state.refresh_z_vals()
    - estrae a caso due categorie distinte zk e zl
    - raccoglie tutti gli archi che hanno peso zk oppure zl
    - unisce questi archi in un unico insieme
    - cerca un nuovo valore z_new comune per tutti gli archi uniti, usando una random bisection search
    - accetta la mossa solo se il guadagno di MDL è positivo
    - in caso di accettazione, assegna z_new a tutti gli archi coinvolti e aggiorna z_vals
    
    input params:
    -> state: stato corrente di tipo W_state
    -> rng: generatore di numeri casuali
    -> nsteps: numero di iterazioni della random bisection per ogni tentativo
    -> max_out: numero di tentativi indipendenti nella ricerca del nuovo valore z_new
    -> max_tries: numero massimo di tentativi per trovare una mossa di merge valida
    -> accept_zero: se False, non accetta valori di categoria pari a zero
    
    output params:
    -> ritorna una tupla (ok, info) dove:
       - ok: True se una mossa di merge è stata accettata, False altrimenti
       - info: dizionario con informazioni sulla mossa accettata, contenente:
               * zk: prima categoria scelta
               * zl: seconda categoria scelta
               * z_new: nuovo valore assegnato dopo il merge
               * gain: guadagno di MDL della mossa
               * n_edges: numero totale di archi coinvolti
               oppure None se non viene accettata nessuna mossa
    
    NB!
    -> Se ci sono meno di 2 categorie di peso presenti, la funzione ritorna subito (False, None).
    """
    # Aggiorno z_vals per sicurezza
    state.refresh_z_vals()
    z_list = state.z_vals

    if len(z_list) < 2:
        return False, None

    # Faccio un loop in cui provo a fare il merge
    for _ in range(max_tries):
        zk, zl = rng.choice(z_list, size=2, replace=False)
        zk = float(zk)
        zl = float(zl)

        # Mi prendo gli archi che hanno valore zk e zl
        edges_k = edges_with_spec_value(state, zk)
        edges_l = edges_with_spec_value(state, zl)

        if (len(edges_k) == 0) or (len(edges_l) == 0):
            continue

        # Unisco gli archi
        edges_merged = list(set(edges_k).union(edges_l))

        z_new, gain = propose_new_category_weight_RB(
            state, 
            edges_k=edges_merged,
            rng=rng,
            nsteps=nsteps,
            max_out=max_out
        )
        
        # Se non trovo niente vado al prossimo ciclo
        if z_new is None or (not np.isfinite(gain)):
            continue

        # Non accetto categorie pari a zero in base al parametro accept_zero
        if not accept_zero:
            if abs(z_new) <= state.tol:
                continue
        # Accetto il cambiamento se migliora
        if gain > 0:
            apply_category_value_change(state, edges_merged, float(z_new))
            state.refresh_z_vals()

            info = {
                "zk": zk,
                "zl": zl,
                "z_new": float(z_new),
                "gain": float(gain),
                "n_edges": int(len(edges_merged))
            }
            return True, info
    return False, None

In [42]:
def split_weight_category_move(
    state: W_state, rng, nsteps=12, max_out=5, max_tries=20, 
    accept_zero=False, allow_existing_category=True, force_distinct=True
):
    """
    Prova una mossa di split di una categoria di peso in due nuove categorie.
    
    La funzione:
    - aggiorna prima la lista delle categorie presenti con state.refresh_z_vals()
    - seleziona tra le categorie che hanno almeno 2 archi associati
    - sceglie una categoria zk da dividere
    - partiziona casualmente i suoi archi in due sottoinsiemi non vuoti, edges_A e edges_B
    - cerca un valore ottimale zA per tutti gli archi in edges_A
    - cerca un valore ottimale zB per tutti gli archi in edges_B
    - valuta il guadagno totale di MDL della mossa di split
    - accetta la mossa solo se il guadagno totale è positivo
    - in caso di accettazione, applica simultaneamente tutti i cambiamenti e aggiorna z_vals
    
    input params:
    -> state: stato corrente di tipo W_state
    -> rng: generatore di numeri casuali
    -> nsteps: numero di iterazioni della random bisection per ogni ricerca di categoria
    -> max_out: numero di tentativi indipendenti nella ricerca di zA e zB
    -> max_tries: numero massimo di tentativi per trovare una mossa di split valida
    -> accept_zero: se False, non accetta valori di categoria pari a zero
    -> allow_existing_category: se False, non accetta valori che coincidono con categorie già presenti
    -> force_distinct: se True, impone che zA e zB siano effettivamente distinti
    
    output params:
    -> ritorna una tupla (ok, info) dove:
       - ok: True se una mossa di split è stata accettata, False altrimenti
       - info: dizionario con informazioni sulla mossa accettata, contenente:
               * zk: categoria iniziale che è stata splittata
               * zA: nuovo valore assegnato al primo sottoinsieme
               * zB: nuovo valore assegnato al secondo sottoinsieme
               * gain: guadagno totale di MDL della mossa
               * mA: numero di archi nel primo sottoinsieme
               * mB: numero di archi nel secondo sottoinsieme
               oppure None se non viene accettata nessuna mossa
    
    NB!
    -> La funzione considera solo categorie con almeno 2 archi
    -> La mossa viene scartata anche se entrambe le nuove categorie coincidono di fatto con la 
    categoria iniziale zk, perché in quel caso non ci sarebbe alcun vero split.
    """

    state.refresh_z_vals()
    z_list = state.z_vals

    # Se non ho categorie non posso fare lo split
    if len(z_list) == 0:
        return False, None

    # Mi prendo tutte le categorie con almeno 2 edges
    splittable = []
    for zk in z_list:
        edges_k = edges_with_spec_value(state, float(zk))
        if len(edges_k) >= 2:
            splittable.append(float(zk))

    if len(splittable) == 0:
        return False, None

    # Ora faccio il loop per provare lo split piu volte finche non trovo una volta che migliora la MLD
    for _ in range(max_tries):
        zk = float(rng.choice(splittable))
        edges_k = edges_with_spec_value(state, zk)
        mk = len(edges_k)

        # Scelgo a caso come splittare la categoria (almeno un edge per ogni nuova categoria)
        s = int(rng.integers(1, mk))
        mask_idx = rng.choice(mk, size=s, replace=False)
        
        edges_A = [edges_k[ii] for ii in mask_idx]
        setA = set(edges_A)
        edges_B = [e for e in edges_k if e not in setA]

        if len(edges_A) == 0 or len(edges_B) == 0:
            continue

        # Ottimizzo zA

        zA, temp_gainA = propose_new_category_weight_RB(
            state,
            edges_k=edges_A,
            rng=rng,
            nsteps=nsteps,
            max_out=max_out
        )
        if zA is None or (not np.isfinite(temp_gainA)):
            continue
        zA = float(zA)

        # Applico i vari filtri a seconda dei parametri passati
        if (not accept_zero) and (abs(zA) <= state.tol):
            continue
        if (not allow_existing_category) and state.is_in_z(zA):
            continue

        # Ottimizzo zB

        zB, temp_gainB = propose_new_category_weight_RB(
            state,
            edges_k=edges_B,
            rng=rng,
            nsteps=nsteps,
            max_out=max_out
        )
        if zB is None or (not np.isfinite(temp_gainB)):
            continue
        zB = float(zB)
        
        # Applico i vari filtri a seconda dei parametri passati
        if (not accept_zero) and (abs(zB) <= state.tol):
            continue
        if (not allow_existing_category) and state.is_in_z(zB):
            continue

        if force_distinct and (abs(zA - zB) <= state.tol):
            continue

        if (abs(zA - zk) <= state.tol) and (abs(zB - zk) <= state.tol):
            continue
        # Valuto il guadagno totale
        changes_tot = [(i, j, zA) for (i, j) in edges_A] + [(i, j, zB) for (i, j) in edges_B]

        gain_tot = delta_MDL(state, changes_tot)

        if not np.isfinite(gain_tot):
            continue

        if gain_tot > 0:
            apply_changes(state, changes_tot)
            state.refresh_z_vals()

            info = {
                    "zk": float(zk),
                    "zA": float(zA),
                    "zB": float(zB),
                    "gain": float(gain_tot),
                    "mA": int(len(edges_A)),
                    "mB": int(len(edges_B))      
            }
            return True, info
    return False, None

In [43]:
# Funzione che effettua un merge e poi uno split e accetta solo se la mossa migliora MDL

def merge_split_weight_category_move(
    state: W_state, rng, nsteps=12, max_out=5, max_tries=20, accept_zero=False,
    allow_existing_category=True
):
    """
    Prova una mossa di merge-split tra due categorie di peso scelte casualmente.
    
    La funzione:
    - aggiorna prima la lista delle categorie presenti con state.refresh_z_vals()
    - estrae a caso due categorie distinte zk e zl
    - raccoglie tutti gli archi che hanno peso zk oppure zl
    - unisce questi archi in un unico insieme
    - partiziona casualmente l'insieme unito in due sottoinsiemi non vuoti, edges_A e edges_B
    - cerca un valore ottimale zA per tutti gli archi in edges_A
    - cerca un valore ottimale zB per tutti gli archi in edges_B
    - valuta il guadagno totale di MDL della mossa
    - accetta la mossa solo se il guadagno totale è positivo
    - in caso di accettazione, applica simultaneamente tutti i cambiamenti e aggiorna z_vals
    
    input params:
    -> state: stato corrente di tipo W_state
    -> rng: generatore di numeri casuali
    -> nsteps: numero di iterazioni della random bisection per ogni ricerca di categoria
    -> max_out: numero di tentativi indipendenti nella ricerca di zA e zB
    -> max_tries: numero massimo di tentativi per trovare una mossa di merge-split valida
    -> accept_zero: se False, non accetta valori di categoria pari a zero
    -> allow_existing_category: se False, non accetta valori che coincidono con categorie già presenti
    
    output params:
    -> ritorna una tupla (ok, info) dove:
       - ok: True se una mossa di merge-split è stata accettata, False altrimenti
       - info: dizionario con informazioni sulla mossa accettata, contenente:
               * zk: prima categoria scelta
               * zl: seconda categoria scelta
               * zA: nuovo valore assegnato al primo sottoinsieme
               * zB: nuovo valore assegnato al secondo sottoinsieme
               * gain: guadagno totale di MDL della mossa
               * mA: numero di archi nel primo sottoinsieme
               * mB: numero di archi nel secondo sottoinsieme
               * m: numero totale di archi coinvolti
               oppure None se non viene accettata nessuna mossa
    
    NB!
    -> Se ci sono meno di 2 categorie di peso presenti, la funzione ritorna subito (False, None).
    """
    state.refresh_z_vals()
    z_list = state.z_vals

    if len(z_list) < 2:
        return False, None

    for _ in range(max_tries):
        # Scelgo a caso due categorie
        zk, zl = rng.choice(z_list, size=2, replace=False)
        zk, zl = float(zk), float(zl)

        # Salvo gli edge che hanno quel valore di peso
        edges_k = edges_with_spec_value(state, zk)
        edges_l = edges_with_spec_value(state, zl)

        if len(edges_k) == 0 or len(edges_l) == 0:
            continue

        edges_merged = list(set(edges_k).union(edges_l))
        m = len(edges_merged)
        if m < 2:
            continue

        # Ora faccio la partizione edges_merged -> due categorie nuove A e B
        s = int(rng.integers(1, m))
        mask_idx = rng.choice(m, size=s, replace=False)

        edges_A = [edges_merged[ii] for ii in mask_idx]
        setA = set(edges_A)
        edges_B = [e for e in edges_merged if e not in setA]

        if len(edges_A) == 0 or len(edges_B) == 0:
            continue

        # Trovo un peso ottimale per la categoria A
        zA, tempgainA = propose_new_category_weight_RB(
            state, edges_k=edges_A, rng=rng, nsteps=nsteps, max_out=max_out
        )

        if zA is None or (not np.isfinite(tempgainA)):
            continue
        zA = float(zA)

        if (not accept_zero) and (abs(zA) <= state.tol):
            continue
        if (not allow_existing_category) and state.is_in_z(zA):
            continue

        # Trovo un peso ottimale per la categoria B
        zB, tempgainB = propose_new_category_weight_RB(
            state, edges_k=edges_B, rng=rng, nsteps=nsteps, max_out=max_out
        )

        if zB is None or (not np.isfinite(tempgainB)):
            continue
        zB = float(zB)
        
        if abs(zA - zB) <= state.tol:
            continue

        

        if (not accept_zero) and (abs(zB) <= state.tol):
            continue
        if (not allow_existing_category) and state.is_in_z(zB):
            continue

        # Infine calcolo il guadagno totale
        changes_tot = [(i, j, zA) for (i, j) in edges_A] + [(i, j, zB) for (i, j) in edges_B]
        gain_tot = delta_MDL(state, changes_tot)

        if not np.isfinite(gain_tot):
            continue

        if gain_tot > 0:
            apply_changes(state, changes_tot)
            state.refresh_z_vals()

            info = {
                "zk": zk,
                "zl": zl,
                "zA": zA,
                "zB": zB,
                "gain": float(gain_tot),
                "mA": int(len(edges_A)),
                "mB": int(len(edges_B)),
                "m": int(m)
            }
            return True, info
    return False, None

In [44]:
def weight_category_distribution_sweep(
    state: W_state,
    rng,
    nsteps=12,
    max_out=5,
    max_tries=20,
    accept_zero=False,
    allow_existing_category=True,
    force_distinct=True,
):
    """
    Esegue uno sweep di aggiornamento della distribuzione delle categorie di peso.
    
    La funzione:
    - prova nell'ordine una mossa di merge, una di split e una di merge-split
    - esegue ciascuna mossa solo se applicabile nello stato corrente
    - accetta ogni mossa solo se il relativo guadagno di MDL è positivo
    - conta quante mosse sono state effettivamente provate
    - conta quante mosse sono state effettivamente accettate
    
    input params:
    -> state: stato corrente di tipo W_state
    -> rng: generatore di numeri casuali
    -> nsteps: numero di iterazioni della random bisection per ogni ricerca di categoria
    -> max_out: numero di tentativi indipendenti nella ricerca dei nuovi valori di categoria
    -> max_tries: numero massimo di tentativi interni per ciascuna mossa
    -> accept_zero: se False, non accetta valori di categoria pari a zero
    -> allow_existing_category: se False, non accetta valori che coincidono con categorie già presenti
    -> force_distinct: se True, impone che nello split le due nuove categorie siano effettivamente distinte
    
    output params:
    -> ritorna una tupla (acc, tried) dove:
       - acc: numero di mosse accettate nello sweep
       - tried: numero di mosse effettivamente provate nello sweep
    
    NB!
    -> Le tre mosse vengono tentate in quest'ordine: merge, poi split, poi merge-split.
    """

    info = {
        "merge": None,
        "split": None,
        "merge_split": None,
    }

    acc = 0
    tried = 0

    # MERGE
    if len(state.z_vals) >= 2:
        tried += 1
        ok, _ = merge_weight_category_move(
            state,
            rng=rng,
            nsteps=nsteps,
            max_out=max_out,
            max_tries=max_tries,
            accept_zero=accept_zero
        )
        if ok:
            acc += 1

    # SPLIT
    if len(state.z_vals) >= 1:
        tried += 1
        ok, _ = split_weight_category_move(
            state,
            rng=rng,
            nsteps=nsteps,
            max_out=max_out,
            max_tries=max_tries,
            accept_zero=accept_zero,
            allow_existing_category=allow_existing_category,
            force_distinct=force_distinct
        )
        if ok:
            acc += 1

    # MERGE-SPLIT
    if len(state.z_vals) >= 2:
        tried += 1
        ok, _ = merge_split_weight_category_move(
            state,
            rng=rng,
            nsteps=nsteps,
            max_out=max_out,
            max_tries=max_tries,
            accept_zero=accept_zero,
            allow_existing_category=allow_existing_category
        )
        if ok:
            acc += 1
    return acc, tried

In [46]:
def network_reconstruction(state: W_state, rng, W_true, min_tries=5, max_tries=10, accepted_gain=1e-2, print_values=True):
    """
    Esegue la procedura completa di ricostruzione della rete tramite sweep successivi
    delle diverse mosse dell'algoritmo.
    
    La funzione:
    - seleziona a ogni iterazione gli archi candidati con peso nullo
    - costruisce l'insieme E degli archi su cui lavorare
    - esegue in sequenza:
      * edge update
      * edge move
      * edge swap
      * weight category value sweep
      * weight category distribution sweep
    - aggiorna i contatori delle mosse accettate
    - calcola a ogni iterazione la Jaccard similarity binarizzata e continua rispetto a W_true
    - controlla un criterio di arresto basato sulla variazione della Jaccard continua
    - aggiorna infine le categorie di peso presenti con state.refresh_z_vals()
    
    input params:
    -> state: stato corrente di tipo W_state
    -> rng: generatore di numeri casuali
    -> W_true: matrice dei pesi vera, usata per valutare la qualità della ricostruzione
    -> min_tries: numero minimo di iterazioni da eseguire prima di poter applicare il criterio di arresto
    -> max_tries: numero massimo di iterazioni dello schema di ricostruzione
    -> accepted_gain: soglia sul cambiamento della Jaccard continua sotto cui la procedura si arresta
    -> print_values: se True, stampa a schermo informazioni diagnostiche a ogni iterazione
    
    output params:
    -> ritorna una tupla (state, info) dove:
       - state: stato finale della ricostruzione
       - info: dizionario contenente:
               * Binarized_jaccard: ultima Jaccard similarity binarizzata calcolata
               * Continuous_jaccard: ultima Jaccard similarity continua calcolata
               * Edge_updates: numero totale di edge updates accettati
               * Edge_moves: numero totale di edge moves accettati
               * Edge_swaps: numero totale di edge swaps accettati
               * Category_value: numero totale di update del valore delle categorie accettati
               * Category_distribution: numero totale di mosse sulla distribuzione delle categorie accettate
    
    NB!
    -> Il criterio di arresto anticipato viene controllato confrontando la Jaccard continua corrente
       con quella dell'iterazione precedente.
    -> Se il cambiamento della Jaccard continua è minore o uguale a accepted_gain e sono già state
       eseguite abbastanza iterazioni (_ > min_tries), la funzione termina prima di max_tries.
    """
    counter_updates = 0
    counter_moves = 0
    counter_swaps = 0
    counter_cat_val = 0
    counter_dist = 0
    info = {}
    for _ in range(max_tries):
        # Scelgo i migliori candidati con peso zero
        candidate_edges = choose_best_candidate_edges(state, rng)
        # Li unisco con i pesi gia diversi da zero
        E_list = build_E_list(state, candidate_edges)


        W_0 = state.W.copy()
        
        acc_upd = edge_update_sweep(state, E_list, rng=rng)
        
        acc_move, __ = edge_move_sweep(state, E_list=E_list, rng=rng)
    
        acc_swap, __ = edge_swap_sweep(state, rng=rng)
    
        acc_cat, __ = weight_category_value_sweep(state, rng=rng)
    
        acc_dist, __ = weight_category_distribution_sweep(state, rng=rng)
    
        
        counter_updates += int(acc_upd)
        counter_moves += int(acc_move)
        counter_swaps += int(acc_swap)
        counter_cat_val += int(acc_cat)
        counter_dist += int(acc_dist)

        
        JcBin = jaccard_similarity(binarize_matrix(W_true, state.tol), binarize_matrix(state.W, state.tol))
        JcCont = jaccard_similarity(W_true, state.W)

        
        
        if _ > 1:
            if print_values:
                print("JcCont - JcCont_old: ", JcCont - JcCont_old)
            if abs(JcCont - JcCont_old) <= accepted_gain and _ > min_tries:
                #print(">>> STOP QUI <<<", "iter =", _, "diff =", abs(JcCont - JcCont_old))
                return state, info


        JcBin_old = JcBin
        JcCont_old = JcCont
        
        W_change = state.W - W_0

        if print_values:
            print("Cambiamenti fatti")
            print(np.sum(np.abs(W_change) > state.tol))
            print("Jacc similarity binarizzata: ", JcBin)
            print("Jaccard similarity continua: ", JcCont)
        state.refresh_z_vals()

        
        info = {
            "Binarized_jaccard": JcBin,
            "Continuous_jaccard": JcCont,
            "Edge_updates": counter_updates,
            "Edge_moves": counter_moves,
            "Edge_swaps": counter_swaps,
            "Category_value": counter_cat_val,
            "Category_distribution": counter_dist
        }
    return state, info